<a href="https://colab.research.google.com/github/ortegatiago/covid19_analysis/blob/master/covid19_prediction_phophet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**COVID 19 Predictions using python Facebook Prophet**

Author: Ortega, Tiago

email: ortega.tiago@gmail.com

github: https://github.com/ortegatiago

linkedin: https://www.linkedin.com/in/ortegatiago/

Facebook Prophet Documentations

https://facebook.github.io/prophet/docs/quick_start.html#python-api


https://facebook.github.io/prophet/


In [ ]:
import pandas as pd
from fbprophet import Prophet